In [1]:
import h5py
import show_params3
import numpy as np
import pandas as pd

from IPython.display import display, Latex, Markdown
from progressbar import ProgressBar

In [2]:
d='/data17/grenache/aalazar/FIRE/GVB/m12b_res7100/output/hdf5/snapdir_172/'
fname=d+'snapshot_172.0.hdf5'

## Exploring the keys

In [3]:
with h5py.File(fname) as f:
    print(f.keys())
    print('')
    print(f['Header'].attrs.keys())
    print('')
    for txt,i in zip(['Gas','DM','Star'],[0,1,4]):
        display(Markdown('**{} keys:**'.format(txt)))
        print(*f['PartType{}'.format(i)].keys(),sep='\n')

<KeysViewHDF5 ['Header', 'PartType0', 'PartType1', 'PartType2', 'PartType4']>

<KeysViewHDF5 ['BoxSize', 'CompactLevel', 'Compactify_Version', 'Flag_Cooling', 'Flag_DoublePrecision', 'Flag_Feedback', 'Flag_IC_Info', 'Flag_Metals', 'Flag_Sfr', 'Flag_StellarAge', 'HubbleParam', 'MassTable', 'NumFilesPerSnapshot', 'NumPart_ThisFile', 'NumPart_Total', 'NumPart_Total_HighWord', 'Omega0', 'OmegaLambda', 'ReadMe', 'Redshift', 'Time']>



**Gas keys:**

Coordinates
Density
ElectronAbundance
InternalEnergy
Masses
Metallicity
NeutralHydrogenAbundance
ParticleChildIDsNumber
ParticleIDGenerationNumber
ParticleIDs
Potential
SmoothingLength
StarFormationRate
Velocities


**DM keys:**

Coordinates
Masses
ParticleChildIDsNumber
ParticleIDGenerationNumber
ParticleIDs
Potential
Velocities


**Star keys:**

Coordinates
Masses
Metallicity
ParticleChildIDsNumber
ParticleIDGenerationNumber
ParticleIDs
Potential
StellarFormationTime
Velocities


In [4]:
show_params3.show(fname)

['Header', 'PartType0', 'PartType1', 'PartType2', 'PartType4']

redshift: 2.0000003196611424
box size: 60000.0 kpc/h or Mpc/h
mass table: [0. 0. 0. 0. 0. 0.]
Omega0: 0.272
OmegaLambda: 0.728
HubbleParam: 0.702

Hi-res particles
[2.4697101e-06 2.4697101e-06 2.4697101e-06 ... 2.4697101e-06 2.4697101e-06
 2.4697101e-06]
[[ 9667.07747464  9989.06252581  9199.64151069]
 [ 9667.11410133  9988.88174719  9199.30714734]
 [ 9666.87346865  9987.8002904   9198.48555853]
 ...
 [ 9590.1125678   9780.19022375 10038.84693958]
 [ 9590.21615706  9780.2743317  10038.98743776]
 [ 9589.60649243  9781.01562356 10038.86663918]]


In [5]:
with h5py.File(fname) as snap:
    print(snap['PartType4'].keys())

<KeysViewHDF5 ['Coordinates', 'Masses', 'Metallicity', 'ParticleChildIDsNumber', 'ParticleIDGenerationNumber', 'ParticleIDs', 'Potential', 'StellarFormationTime', 'Velocities']>


## Using what Francisco gave me

In [6]:
def get_data(filename,num_of_file,key1,key2):
    if num_of_file == 1:
        f = h5py.File(filename+'.hdf5', 'r')
        if key1 == 'Header':
            return f[key1].attrs[key2]
        else:
            return f[key1][key2][:]
    else:
        for i in range(0,num_of_file):
            f = h5py.File(filename+'.'+str(i)+'.hdf5', 'r')
            if key1 == 'Header':
                return f[key1].attrs[key2]
            else:
                if ( len(f[key1][key2][:].shape)==1 ):
                    if i==0:
                        result = f[key1][key2][:]
                    else:
                        result = np.hstack( (result,f[key1][key2][:]) )
                else:
                    if i==0:
                        result = f[key1][key2][:]
                    else:
                        result = np.vstack( (result,f[key1][key2][:]) )
        return result

# Creating Dataset for MLS

In [7]:
#metal=get_data(d+'snapshot_172',8,'PartType0','Metallicity')
#coords=get_data(d+'snapshot_172',8,'PartType0','Velocities')

#print('{0:0.4e} gas particles?'.format(metal.shape[0]))
#print(metal.shape)
#print(coords.shape)

In [9]:
def mlsdat():
    p='PartType0'
    data=get_data(d+'snapshot_172',8,p,'Coordinates')
    with h5py.File(fname) as f:        
        pbar=ProgressBar()
        keys=list(f[p].keys())
        for key in pbar(keys[1:]):
            new=get_data(d+'snapshot_172',8,p,key)
            s=new.shape
            if len(s)==1:
                new=new.reshape(-1,1)
            data=np.append(data,new,axis=1)
    i_s=list(np.arange(len(data)))
    i_s=np.random.choice(i_s,size=31250,replace=False)
    return data[i_s]

In [35]:
def labels():
    p='PartType0'
    labs=['x','y','z']
    with h5py.File(fname) as f:
        keys=list(f[p].keys())
        pbar=ProgressBar()
        for key in pbar(keys[1:]):
            new=get_data(d+'snapshot_172',2,p,key)
            s=new.shape
            if len(s)==1:
                labs+=[key]
            else:
                labs+=[key+str(i) for i in range(s[1])]
    return labs
labs=labels()

100% |########################################################################|


In [36]:
labs

['x',
 'y',
 'z',
 'Density',
 'ElectronAbundance',
 'InternalEnergy',
 'Masses',
 'Metallicity0',
 'Metallicity1',
 'Metallicity2',
 'Metallicity3',
 'Metallicity4',
 'Metallicity5',
 'Metallicity6',
 'Metallicity7',
 'Metallicity8',
 'Metallicity9',
 'Metallicity10',
 'Metallicity11',
 'Metallicity12',
 'Metallicity13',
 'Metallicity14',
 'NeutralHydrogenAbundance',
 'ParticleChildIDsNumber',
 'ParticleIDGenerationNumber',
 'ParticleIDs',
 'Potential',
 'SmoothingLength',
 'StarFormationRate',
 'Velocities0',
 'Velocities1',
 'Velocities2']

In [10]:
trimdat=mlsdat()

100% |########################################################################|


In [39]:
df=pd.DataFrame(trimdat,columns=labs)
df.to_hdf('m12b_res7100_s172.h5',index=True,key='df')

In [41]:
#with h5py.File(fname) as f:  
#    df2=pd.read_hdf('m12b_res7100_s172.h5')
df2=pd.read_hdf('m12b_res7100_s172.h5')

In [42]:
df2

,x,y,z,Density,ElectronAbundance,InternalEnergy,Masses,Metallicity0,Metallicity1,Metallicity2,...,NeutralHydrogenAbundance,ParticleChildIDsNumber,ParticleIDGenerationNumber,ParticleIDs,Potential,SmoothingLength,StarFormationRate,Velocities0,Velocities1,Velocities2
0,29917.358889,29738.340306,28767.629915,8.213907e-10,1.166476,167.317902,4.961228e-07,0.000002,0.250003,3.260000e-07,...,3.222653e-06,0.0,0.0,74056093.0,81498.554688,16.648146,0.0,-287.150665,-51.116577,-171.829971
1,28494.974861,29880.112043,29276.178778,1.107154e-08,1.165655,650.933899,4.961228e-07,0.000002,0.250003,3.260000e-07,...,1.590233e-05,0.0,0.0,26435134.0,74773.445312,6.995638,0.0,-74.553932,-119.218933,-214.816544
2,28427.544710,29063.310240,27569.071711,2.803810e-09,1.166309,409.099182,4.961228e-07,0.000002,0.250003,3.260000e-07,...,5.745126e-06,0.0,0.0,16700518.0,69539.031250,11.056781,0.0,-120.383331,-48.767429,-156.098511
3,28814.811754,29832.392930,29016.404052,9.544530e-09,1.166374,2637.820801,4.961228e-07,0.000002,0.250003,3.304355e-07,...,3.568064e-06,0.0,0.0,40898840.0,71292.210938,7.362494,0.0,-133.621704,-222.870834,-197.545319
4,29080.344675,30150.698148,28019.456256,5.834753e-09,1.166345,1304.536255,4.961228e-07,0.000012,0.250010,1.189796e-06,...,4.677135e-06,0.0,0.0,45919747.0,73344.531250,8.624970,0.0,-189.470322,-82.289154,-169.699371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31245,28822.662511,29910.229757,29203.779110,2.855902e-08,1.164526,864.348816,4.961228e-07,0.000002,0.250003,3.260000e-07,...,3.196423e-05,0.0,0.0,60308513.0,74379.265625,5.081618,0.0,-283.957428,-189.545731,-286.217712
31246,27851.430562,28730.788912,29236.923087,2.380337e-08,1.164669,736.369080,4.961228e-07,0.000002,0.250003,3.260000e-07,...,3.082541e-05,0.0,0.0,7564101.0,74019.648438,5.422717,0.0,-63.409203,3.636941,-175.944565
31247,29352.786135,29391.488992,28920.299082,2.508467e-09,1.166847,32589.341797,4.961228e-07,0.000225,0.250207,2.758144e-05,...,7.008859e-08,0.0,0.0,70472876.0,76786.421875,11.483388,0.0,134.640411,-484.663910,-61.588097
31248,28026.248056,29677.980243,28149.250528,5.380688e-10,1.166522,134.247742,4.961228e-07,0.000002,0.250003,3.260000e-07,...,2.464534e-06,0.0,0.0,6723973.0,71812.429688,19.164244,0.0,-14.965051,-79.248169,-119.321037
